In [1]:
from EventStreamData import *
from GUID import *
import pandas as pd 
import numpy as np 
from pandas.io.json import json_normalize

In [2]:
gameresult = GameResult().get_data()

In [3]:
gamestart = GameStart().get_data()

In [4]:
display(gameresult)

,esports_match_id,num_map,map_name,map_type,start_time,end_time,total_game_time,map_winner,team_one_name,team_two_name,team_one_score,team_two_score
0,35509,1,Oasis,CONTROL,1602155640554,1602156350366,705.092,San Francisco Shock,Seoul Dynasty,San Francisco Shock,0,2
1,35509,2,Hollywood,HYBRID,1602156749404,1602157600378,846.381,San Francisco Shock,Seoul Dynasty,San Francisco Shock,0,3
2,35509,3,Volskaya Industries,ASSAULT,1602158703471,1602159412981,704.915,Seoul Dynasty,San Francisco Shock,Seoul Dynasty,1,2
3,35509,4,Havana,PAYLOAD,1602159881012,1602160675575,789.969,Seoul Dynasty,San Francisco Shock,Seoul Dynasty,0,3
4,35509,5,Busan,CONTROL,1602161154558,1602161757673,598.395,San Francisco Shock,Seoul Dynasty,San Francisco Shock,0,2
5,35510,1,Busan,CONTROL,1602163542926,1602165314628,848.179,Shanghai Dragons,Philadelphia Fusion,Shanghai Dragons,1,2
6,35510,2,King's Row,HYBRID,1602165877018,1602166320954,439.345,Shanghai Dragons,Shanghai Dragons,Philadelphia Fusion,1,0
7,35510,3,Volskaya Industries,ASSAULT,1602167872266,1602168393378,516.518,Shanghai Dragons,Shanghai Dragons,Philadelphia Fusion,1,0
8,35551,1,Oasis,CONTROL,1602234585825,1602235233937,643.392,San Francisco Shock,San Francisco Shock,Shanghai Dragons,2,0
9,35551,2,Hollywood,HYBRID,1602235875309,1602236949719,1069.815,San Francisco Shock,Shanghai Dragons,San Francisco Shock,2,3
